In [40]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
import torch
torch.cuda.set_device(1)

In [2]:
path = untar_data(URLs.CIFAR)

In [3]:
data = ImageDataBunch.from_folder(path, train = 'train', valid = 'test', bs = 128)
data.normalize(cifar_stats)

ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
bird,bird,bird,bird,bird
Path: /home/akshay/.fastai/data/cifar10;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
bird,bird,bird,bird,bird
Path: /home/akshay/.fastai/data/cifar10;

Test: None

In [7]:
learn = cnn_learner(data, models.resnet34, metrics = accuracy)
learn = learn.load('bs128')
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 16, 16]         9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 16, 16]         128        True      
______________________________________________________________________
ReLU                 [64, 16, 16]         0          False     
______________________________________________________________________
MaxPool2d            [64, 8, 8]           0          False     
______________________________________________________________________
Conv2d               [64, 8, 8]           36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 8, 8]           128        True      
______________________________________________________________________
ReLU                 [64, 8, 8]           0          False     
___________________________________________________

In [8]:
class SaveFeatures :
    def __init__(self, m) : 
        self.handle = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, m, inp, outp) : 
        self.features = outp
    def remove(self) :
        self.handle.remove()

In [46]:
net = learn.model
net[0]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [23]:
# saving outputs of all Basic Blocks
sf = [SaveFeatures(m) for m in [net[0][4], net[0][5], net[0][6], net[0][7]]]

In [24]:
# sample data
x, y = next(iter(data.train_dl))

In [30]:
img1 = x[None, 1]
img1.shape

torch.Size([1, 3, 32, 32])

In [43]:
net(torch.autograd.Variable(img1))

tensor([[-5.4855, -4.1845,  2.2016,  2.6475,  2.6617,  4.4456, -2.3378,  2.0481,
         -7.1660, -3.4118]], device='cuda:1', grad_fn=<AddmmBackward>)

In [45]:
[o.features.shape for o in sf]

[torch.Size([1, 64, 8, 8]),
 torch.Size([1, 128, 4, 4]),
 torch.Size([1, 256, 2, 2]),
 torch.Size([1, 512, 1, 1])]